In [ ]:
from simulated_annealing.algorithm import simple_simulated_annealing, SAResult
from simulated_annealing.neighbors_generator import generate_2opt_neighbor, generate_2swap_neighbor

import pandas as pd
import numpy as np
from numpy.random import default_rng
from typing import Callable, List, TypeVar, Tuple
import seaborn as sns
import matplotlib.pylab as plt

T = TypeVar("T")
REPS = 10

In [ ]:
def shape_progress(result: SAResult) -> pd.DataFrame:
    return pd.DataFrame({
        "Iteration": np.arange(len(result.b_progress)),
        "Best cost": result.b_progress,
        "Current cost": result.c_progress,
        "Acceptance rate": result.acceptance_rate,
    })


def shuffle(lst: List[T]) -> List[T]:
    rng = default_rng()
    lst_cp = lst.copy()
    rng.shuffle(lst_cp)
    return lst_cp


def generate_path_distance_computer(fp: str) -> Tuple[Callable[[List[int]], float], int]:
    data = np.loadtxt(fp)
    n = data.shape[0]

    def compute_pd(path: List[int]) -> float:
        return np.sum([data[path[i], path[(i + 1) % n]] for i in range(n)])

    return compute_pd, n


def experiment(data_set: str,
               nbr: Callable[[List[int]], List[int]],
               name: str,
               max_iters=10_000,
               seeds=np.random.randint(100, size=REPS),
               temp_factor=0.95,
               initial_temp=10_000) -> pd.DataFrame:
    obj_f, n = generate_path_distance_computer(data_set)
    results = []

    for i in range(REPS):
        np.random.seed(seeds[i])
        s0 = shuffle([i for i in range(n)])
        results.append(
            simple_simulated_annealing(obj_f, nbr, s0,
                                       temp_factor=temp_factor,
                                       initial_temperature=initial_temp,
                                       max_iterations=max_iters)
        )

    df = pd.concat([shape_progress(r) for r in results])
    df["Neighbor method"] = name
    return df

In [ ]:
def plot(df: pd.DataFrame, optimal: float) -> Tuple[plt.Figure, Tuple]:
    fig, (a1, a2, a3) = plt.subplots(1, 3, width_ratios=[0.3, 0.35, 0.35], figsize=(10, 3))
    sns.lineplot(df, x="Iteration", y="Acceptance rate", ax=a1)
    a2.axhline(optimal, color="black")
    sns.lineplot(df, x="Iteration", y="Best cost", hue="Neighbor method", ax=a2)
    a3.axhline(optimal, color="black")
    sns.lineplot(df, x="Iteration", y="Current cost", hue="Neighbor method", ax=a3)
    
    return fig, (a1, a2, a3)

In [ ]:
gr17_2swap_df = experiment("data/gr17_d.txt", generate_2swap_neighbor, "2-swap",
                           max_iters=1500, initial_temp=1000, temp_factor=0.99)
gr17_2opt_df = experiment("data/gr17_d.txt", generate_2opt_neighbor, "2-opt",
                          max_iters=1500, initial_temp=1000, temp_factor=0.99)
gr17_df = pd.concat([gr17_2swap_df, gr17_2opt_df])
gr17_df

In [ ]:
gr17_fig, _ = plot(gr17_df, 2085)
gr17_fig.savefig("../images/gr17_sa", dpi=600, bbox_inches="tight")

In [ ]:
fri26_2swap_df = experiment("data/fri26_d.txt", generate_2swap_neighbor, "2-swap",
                           max_iters=2500, initial_temp=1000, temp_factor=0.99)
fri26_2opt_df = experiment("data/fri26_d.txt", generate_2opt_neighbor, "2-opt",
                          max_iters=2500, initial_temp=1000, temp_factor=0.99)
fri26_df = pd.concat([gr17_2swap_df, gr17_2opt_df])
fri26_df

In [ ]:
fri26_fig, _ = plot(fri26_df, 937)
fri26_fig.savefig("../images/fri26_sa", dpi=600, bbox_inches="tight")

In [ ]:
att48_2swap_df = experiment("data/fri26_d.txt", generate_2swap_neighbor, "2-swap",
                           max_iters=2500, initial_temp=1000, temp_factor=0.99)
att48_2opt_df = experiment("data/fri26_d.txt", generate_2opt_neighbor, "2-opt",
                          max_iters=2500, initial_temp=1000, temp_factor=0.99)
att48_df = pd.concat([gr17_2swap_df, gr17_2opt_df])
att48_df

In [ ]:
att48_fig, _ = plot(att48_df, 33523)
att48_fig.savefig("../images/att48_sa", dpi=600, bbox_inches="tight")